In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

### Initialize Spark Session

In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("BigDataAnalyticsPipeline").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/09 23:25:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Large Data Ingestion

In [3]:
# Define the base directory for the project
base_dir = "./data"

# Gather paths to all text files that are actually JSON
file_paths = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        # Add only text files (assuming they don’t have a .json extension)
        if not file.endswith('.json'):
            file_paths.append(os.path.join(root, file))

# Load all JSON text files into a single Spark DataFrame
# Using the option `multiLine=True` if each file contains multiple JSON objects across lines
df = spark.read.option("multiLine", True).json(file_paths)

# Display schema and sample records to verify
df.printSchema()
df.show(5)


24/11/09 23:25:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
24/11/09 23:26:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- abstracts-retrieval-response: struct (nullable = true)
 |    |-- affiliation: string (nullable = true)
 |    |-- authkeywords: struct (nullable = true)
 |    |    |-- author-keyword: string (nullable = true)
 |    |-- authors: struct (nullable = true)
 |    |    |-- author: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- @_fa: string (nullable = true)
 |    |    |    |    |-- @auid: string (nullable = true)
 |    |    |    |    |-- @seq: string (nullable = true)
 |    |    |    |    |-- affiliation: string (nullable = true)
 |    |    |    |    |-- author-url: string (nullable = true)
 |    |    |    |    |-- ce:alias: string (nullable = true)
 |    |    |    |    |-- ce:alt-name: string (nullable = true)
 |    |    |    |    |-- ce:degrees: string (nullable = true)
 |    |    |    |    |-- ce:given-name: string (nullable = true)
 |    |    |    |    |-- ce:indexed-name: string (nullable = true)
 |    |    |    |    |

+----------------------------+
|abstracts-retrieval-response|
+----------------------------+
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
+----------------------------+
only showing top 5 rows



### Data Transformation

In [22]:
# Extract title, abstract, publication date, and affiliation
selected_df = df.select(
    col("abstracts-retrieval-response.coredata.dc:identifier").alias("document_id"), 
    col("abstracts-retrieval-response.coredata.dc:title").alias("title"),
    col("abstracts-retrieval-response.coredata.dc:description").alias("description"),
    col("abstracts-retrieval-response.coredata.prism:coverDate").alias("publication_date"),
    col("abstracts-retrieval-response.coredata.dc:publisher").alias("publisher")
)

# Show the final DataFrame
selected_df.show(5, truncate=False)


+---------------------+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------